# Instalando biblioteca minio python

In [2]:
!pip install minio

  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl.metadata (6.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 863.2 kB/s eta 0:00:00a 0:00:01
Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:0000:0100:01
Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.6/477.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 11.3 MB/s eta 0:00:00


In [1]:
import requests
import csv
import pandas as pd

# Criando Bucket

In [2]:
from minio import Minio

client = Minio(
    "localhost:9000",
    access_key="dev_user_01",
    secret_key="devpassword01",
    secure=False
)

bucket_name = "test-bucket"

found = client.bucket_exists(bucket_name)
if not found:
    client.make_bucket(bucket_name)

## Importando arquivo ao bucket

In [5]:
destination_file = 'test.txt'
source_file = './data/test.txt'

client.fput_object(bucket_name, destination_file, source_file,)

# Instalando Pyspark ao ambiente Jupyter

In [ ]:
#!pip install pyspark

Como esse ambiente de desenvolvimento do projeto está sendo utilizado o conda como administrador de pacotes e ambientes, optaremos por instalar o pyspark via conda

In [6]:
!conda install -c conda-forge pyspark

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0



## Package Plan ##

  environment location: /home/galaxia/anaconda3/envs/pyminio

  added / updated specs:
    - pyspark


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20211102.0      |       h27087fc_1         1.1 MB  conda-forge
    arrow-cpp-14.0.2           |       h374c478_1        11.7 MB
    aws-c-auth-0.6.19          |       h5eee18b_0          99 KB
    aws-c-cal-0.5.20           |       hdbd6064_0          42 KB
    aws-c-common-0.8.5         |       h5eee18b_0         207 KB
    aws-c-compression-0.2.16   |       h5eee18b_0          18 KB
    aws-c-event-

In [3]:
def search_csvs():
    url_list = []
    for i in range(18, 21):
        url_first_semester = f"https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-20{str(i)}-01.csv"
        url_second_semester = f"https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-20{str(i)}-02.csv"
        url_list.append(url_first_semester)
        url_list.append(url_second_semester)
    return url_list

In [4]:
def download_csvs(url_list):
    df_list = []
    for url in url_list:
        try:
            with requests.Session() as s:
                decoded_content = s.get(url).content.decode('utf-8')
                downloaded_csv = csv.reader(decoded_content.splitlines(), delimiter=';')
                df_list.append(pd.DataFrame(list(downloaded_csv)))
        except Exception as e:
            print(f"Error downloading csv: {e}")
    final_df = pd.concat(df_list, ignore_index=True)
    final_df.to_csv('./data/final_df.csv', index=False)        

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col

spark = SparkSession.builder.appName('minio-test').getOrCreate()
web_dataset = search_csvs()
print(web_dataset)
#download_csvs(web_dataset)
dataset = './data/final_df.csv'
gasoline_df = spark.read.format('csv').option('header', 'true').load(dataset)

gasoline_df.count()

24/04/24 00:15:16 WARN Utils: Your hostname, galaxia-Vostro-3520 resolves to a loopback address: 127.0.1.1; using 192.168.10.57 instead (on interface wlp0s20f3)
24/04/24 00:15:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/24 00:15:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


['https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2018-01.csv', 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2018-02.csv', 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2019-01.csv', 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2019-02.csv', 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-01.csv', 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsas/ca/ca-2020-02.csv']


2216503

In [11]:
sc = spark.sparkContext

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "dev_user_01")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "devpassword01")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")

In [14]:
gasoline_df \
    .write \
    .mode("overwrite") \
    .parquet(f"s3a://{bucket_name}/gasoline_state.parquet")

Py4JJavaError: An error occurred while calling o67.parquet.
: java.lang.NoClassDefFoundError: software/amazon/awssdk/awscore/exception/AwsServiceException
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.apache.hadoop.conf.Configuration.getClassByNameOrNull(Configuration.java:2625)
	at org.apache.hadoop.conf.Configuration.getClassByName(Configuration.java:2590)
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2686)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execu